## Beágyazások

Az előző példánkban nagy dimenziójú bag-of-words vektorokkal dolgoztunk, amelyek hossza `vocab_size`, és kifejezetten alacsony dimenziójú pozíciós reprezentációs vektorokat alakítottunk át ritka egyhot reprezentációvá. Ez az egyhot reprezentáció nem memóriahatékony. Emellett minden szót egymástól függetlenül kezelünk, így az egyhot kódolt vektorok nem fejezik ki a szavak közötti szemantikai hasonlóságokat.

Ebben az egységben tovább fogjuk vizsgálni a **News AG** adathalmazt. Kezdjük azzal, hogy betöltjük az adatokat, és előhívunk néhány definíciót az előző egységből.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Mi az az embedding?

Az **embedding** ötlete az, hogy a szavakat alacsonyabb dimenziójú, sűrű vektorokkal ábrázoljuk, amelyek tükrözik a szó szemantikai jelentését. Később megbeszéljük, hogyan lehet értelmes szóbeágyazásokat létrehozni, de most tekintsünk az embeddingre úgy, mint egy módszerre, amely csökkenti egy szóvektor dimenzióját.

Tehát egy embedding réteg egy szót kap bemenetként, és egy meghatározott `embedding_size` méretű kimeneti vektort állít elő. Bizonyos értelemben nagyon hasonlít egy `Dense` rétegre, de ahelyett, hogy egy one-hot kódolt vektort venne bemenetként, képes egy szó számát fogadni.

Ha az embedding réteget használjuk a hálózatunk első rétegeként, akkor áttérhetünk a bag-of-words modellről egy **embedding bag** modellre. Ebben az esetben először minden szót a szövegünkben a megfelelő embeddingre alakítunk, majd valamilyen aggregáló függvényt számítunk ki az összes embedding felett, például `sum`, `average` vagy `max`.

![Kép, amely egy embedding osztályozót mutat öt szekvencia szóra.](../../../../../translated_images/hu/embedding-classifier-example.b77f021a7ee67eee.webp)

Az osztályozó neurális hálózatunk a következő rétegekből áll:

* `TextVectorization` réteg, amely egy sztringet kap bemenetként, és egy token számokat tartalmazó tenzort állít elő. Meghatározunk egy ésszerű `vocab_size` szókészlet méretet, és figyelmen kívül hagyjuk a ritkábban használt szavakat. A bemenet alakja 1 lesz, a kimenet alakja pedig $n$, mivel $n$ tokent kapunk eredményként, amelyek mindegyike 0 és `vocab_size` közötti számokat tartalmaz.
* `Embedding` réteg, amely $n$ számot kap, és minden számot egy adott hosszúságú sűrű vektorra redukál (például 100-ra a példánkban). Így az $n$ alakú bemeneti tenzor $n\times 100$ alakú tenzorrá alakul.
* Aggregációs réteg, amely ennek a tenzornak az átlagát számítja ki az első tengely mentén, azaz kiszámítja az összes $n$ bemeneti tenzor átlagát, amelyek különböző szavakhoz tartoznak. Ennek a rétegnek a megvalósításához egy `Lambda` réteget használunk, és átadjuk neki az átlag kiszámítására szolgáló függvényt. A kimenet alakja 100 lesz, és ez az egész bemeneti szekvencia numerikus reprezentációja.
* Végső `Dense` lineáris osztályozó.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


A `summary` kimenetben az **output shape** oszlopban az első tensor dimenzió `None` a minibatch méretét jelöli, míg a második a token szekvencia hosszát. A minibatch-ben lévő összes token szekvencia hossza eltérő. A következő szekcióban arról lesz szó, hogyan kezeljük ezt.

Most pedig tanítsuk be a hálózatot:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Megjegyzés**: a vektorizálót az adatok egy részhalmazára alapozva építjük. Ez azért történik, hogy felgyorsítsuk a folyamatot, és előfordulhat, hogy nem minden token a szövegünkből szerepel a szókincsben. Ebben az esetben ezek a tokenek figyelmen kívül maradnak, ami kissé alacsonyabb pontosságot eredményezhet. Azonban a valóságban a szöveg egy részhalmaza gyakran jó szókincsbecslést ad.


### Változó szekvenciaméretek kezelése

Nézzük meg, hogyan zajlik a tanítás minibatch-ekben. A fenti példában a bemeneti tenzor dimenziója 1, és 128 hosszúságú minibatch-eket használunk, így a tenzor tényleges mérete $128 \times 1$. Azonban minden mondatban eltérő számú token található. Ha a `TextVectorization` réteget alkalmazzuk egyetlen bemenetre, a visszaadott tokenek száma eltérő lesz, attól függően, hogyan történik a szöveg tokenizálása:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Azonban, amikor a vektorizálót több szekvenciára alkalmazzuk, téglalap alakú tenzort kell előállítania, ezért a nem használt elemeket a PAD tokennel tölti ki (ami esetünkben nulla):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Itt láthatjuk az beágyazásokat:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Megjegyzés**: A kitöltés minimalizálása érdekében bizonyos esetekben érdemes az összes szekvenciát a növekvő hosszúság (pontosabban a tokenek száma) sorrendjében rendezni az adatállományban. Ez biztosítja, hogy minden minibatch hasonló hosszúságú szekvenciákat tartalmazzon.


## Szemantikus beágyazások: Word2Vec

Az előző példánkban a beágyazási réteg megtanulta a szavakat vektorreprezentációkká leképezni, azonban ezek a reprezentációk nem rendelkeztek szemantikai jelentéssel. Jó lenne olyan vektorreprezentációt tanulni, amelyben hasonló szavak vagy szinonimák olyan vektorokhoz tartoznak, amelyek valamilyen vektortávolság (például euklideszi távolság) szempontjából közel vannak egymáshoz.

Ehhez elő kell tanítanunk a beágyazási modellt egy nagy szöveggyűjteményen, például a [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) technika segítségével. Ez két fő architektúrán alapul, amelyeket a szavak elosztott reprezentációjának előállítására használnak:

 - **Folytonos szótáska** (CBoW), ahol a modellt arra tanítjuk, hogy a környező kontextusból megjósoljon egy szót. Az ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ esetén a modell célja, hogy $(W_{-2},W_{-1},W_1,W_2)$ alapján megjósolja $W_0$-t.
 - **Folytonos ugrógram** a CBoW ellentéte. A modell a környező kontextusszavak ablakát használja a jelenlegi szó megjóslására.

A CBoW gyorsabb, míg az ugrógram lassabb, de jobban reprezentálja a ritka szavakat.

![Kép, amely bemutatja a CBoW és az ugrógram algoritmusokat a szavak vektorokká alakításához.](../../../../../translated_images/hu/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Ahhoz, hogy kísérletezzünk a Google News adathalmazon előtanított Word2Vec beágyazással, használhatjuk a **gensim** könyvtárat. Az alábbiakban megkeressük a 'neural' szóhoz leginkább hasonló szavakat.

> **Megjegyzés:** Amikor először hozunk létre szóvektorokat, azok letöltése eltarthat egy ideig!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


A szó vektor beágyazását is kinyerhetjük, hogy a klasszifikációs modell képzéséhez használjuk. A beágyazásnak 300 komponense van, de itt az érthetőség kedvéért csak a vektor első 20 komponensét mutatjuk be:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

A szemantikai beágyazások nagyszerűsége abban rejlik, hogy a vektorkódolást szemantikák alapján lehet manipulálni. Például kérhetjük, hogy találjunk egy olyan szót, amelynek vektoros reprezentációja a lehető legközelebb áll a *király* és *nő* szavakhoz, és a lehető legtávolabb a *férfi* szótól:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

A fenti példa némi belső GenSym varázslatot használ, de az alapvető logika valójában meglehetősen egyszerű. Az érdekesség az embeddingekkel kapcsolatban az, hogy normál vektorműveleteket végezhetünk az embedding vektorokon, és ezek a műveletek a szavak **jelentéseire** vonatkoznak. A fenti példa vektorműveletek formájában fejezhető ki: kiszámítjuk a **KIRÁLY-FÉRFI+NŐ** vektort (a `+` és `-` műveletek a megfelelő szavak vektoros reprezentációin történnek), majd megkeressük a szótárban azt a szót, amelyik a legközelebb áll ehhez a vektorhoz:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Kis együtthatókat kellett hozzáadnunk a *man* és *woman* vektorokhoz – próbáld meg eltávolítani őket, hogy lásd, mi történik.

A legközelebbi vektor megtalálásához a TensorFlow eszközeit használjuk, hogy kiszámítsuk a távolságok vektorát a saját vektorunk és a szókincs összes vektora között, majd az `argmin` segítségével megtaláljuk a minimális szó indexét.


Miközben a Word2Vec nagyszerű módnak tűnik a szavak szemantikájának kifejezésére, számos hátránya van, többek között az alábbiak:

* Mind a CBoW, mind a skip-gram modellek **prediktív beágyazások**, és csak a lokális kontextust veszik figyelembe. A Word2Vec nem használja ki a globális kontextust.
* A Word2Vec nem veszi figyelembe a szavak **morfológiáját**, vagyis azt a tényt, hogy a szó jelentése függhet a szó különböző részeitől, például a gyöktől.

A **FastText** megpróbálja leküzdeni a második korlátozást, és a Word2Vec-re épít azáltal, hogy minden szóhoz és az azon belüli karakter n-gramokhoz vektorreprezentációkat tanul. A reprezentációk értékeit minden tanítási lépésnél egyetlen vektorrá átlagolja. Bár ez jelentősen megnöveli az előképzés számítási igényét, lehetővé teszi, hogy a szóbeágyazások kódolják az al-szó információkat.

Egy másik módszer, a **GloVe**, eltérő megközelítést alkalmaz a szóbeágyazásokhoz, amely a szó-kontextus mátrix faktorizációján alapul. Először egy nagy mátrixot épít, amely megszámolja a szavak előfordulásait különböző kontextusokban, majd megpróbálja ezt a mátrixot alacsonyabb dimenziókban ábrázolni úgy, hogy minimalizálja a rekonstrukciós veszteséget.

A gensim könyvtár támogatja ezeket a szóbeágyazásokat, és kísérletezhetsz velük, ha módosítod a fenti modellbetöltő kódot.


## Előre betanított beágyazások használata Kerasban

Az előző példát módosíthatjuk úgy, hogy a beágyazási réteg mátrixát szemantikai beágyazásokkal, például Word2Vec-kel töltsük fel. Az előre betanított beágyazás és a szövegkorpusz szókészlete valószínűleg nem fog egyezni, ezért választanunk kell egyet. Itt a két lehetséges opciót vizsgáljuk meg: a tokenizáló szókészletének használatát, illetve a Word2Vec beágyazások szókészletének használatát.

### A tokenizáló szókészletének használata

A tokenizáló szókészletének használatakor a szókészlet egyes szavaihoz lesznek megfelelő Word2Vec beágyazások, míg mások hiányozni fognak. Tekintve, hogy a szókészlet mérete `vocab_size`, és a Word2Vec beágyazási vektor hossza `embed_size`, a beágyazási réteget egy `vocab_size`$\times$`embed_size` alakú súlymátrix fogja képviselni. Ezt a mátrixot a szókészleten végighaladva fogjuk feltölteni:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


A Word2Vec szókészletében nem szereplő szavakat nullákkal hagyhatjuk, vagy generálhatunk egy véletlenszerű vektort.

Most definiálhatunk egy beágyazási réteget előre betanított súlyokkal:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Megjegyzés**: Figyeljük meg, hogy az `Embedding` létrehozásakor a `trainable=False` értéket állítottuk be, ami azt jelenti, hogy nem tanítjuk újra az Embedding réteget. Ez kissé alacsonyabb pontosságot eredményezhet, de felgyorsítja a tanítást.

### Az embedding szókincs használata

Az előző megközelítés egyik problémája, hogy a TextVectorization és az Embedding által használt szókincsek eltérőek. Ennek a problémának a megoldására az alábbi megoldások egyikét alkalmazhatjuk:
* Újra betanítjuk a Word2Vec modellt a saját szókincsünkre.
* Betöltjük az adatainkat a Word2Vec előre betanított modell szókincsével. Az adatok betöltésekor megadhatjuk a használandó szókincset.

Az utóbbi megközelítés egyszerűbbnek tűnik, ezért valósítsuk meg. Először is létrehozunk egy `TextVectorization` réteget a Word2Vec embeddingekből származó, megadott szókincs alapján:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

A gensim szóbeágyazási könyvtár tartalmaz egy kényelmes függvényt, `get_keras_embeddings`, amely automatikusan létrehozza a megfelelő Keras beágyazási réteget az Ön számára.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Az egyik oka annak, hogy nem látunk nagyobb pontosságot, az, hogy az adatállományunkból néhány szó hiányzik az előre betanított GloVe szókészletből, és így lényegében figyelmen kívül maradnak. Ennek leküzdésére saját beágyazásokat tudunk tanítani az adatállományunk alapján.


## Kontextuális beágyazások

A hagyományos előre betanított beágyazási reprezentációk, mint például a Word2Vec, egyik fő korlátja az, hogy bár képesek egy szó bizonyos jelentését megragadni, nem tudják megkülönböztetni a különböző jelentéseket. Ez problémákat okozhat az utófeldolgozó modellekben.

Például a "play" szó különböző jelentéssel bír az alábbi két mondatban:
- Elmentem egy **színdarabra** a színházba.
- John **játszani** szeretne a barátaival.

Az általunk tárgyalt előre betanított beágyazások mindkét jelentést ugyanabban a beágyazásban reprezentálják. Ennek a korlátnak a leküzdéséhez olyan beágyazásokat kell létrehoznunk, amelyek a **nyelvi modellre** épülnek, amelyet egy nagy szövegkorpusz alapján tanítottak be, és *tudja*, hogyan lehet a szavakat különböző kontextusokban összekapcsolni. A kontextuális beágyazások részletes tárgyalása túlmutat ennek az oktatóanyagnak a keretein, de visszatérünk rájuk, amikor a következő egységben a nyelvi modellekről beszélünk.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
